In [3]:
%run env_setup.py
import keras
from importlib import reload
from lessdeep.datasets.stanford import imdb

Using TensorFlow backend.


In [4]:
idx = imdb.get_word_index()
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

Use pre-calculated vocabulary


['the', 's', 'and', 'a', 'of', 'to', 'is', 'br', 'it', 'in']

In [5]:
word2idx = {w: i for w, i in idx.items()}

In [ ]:
(x_train, labels_train), (x_test, labels_test) = 